In [19]:
import requests
import bs4 as BeautifulSoup

from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import requests
from bs4 import BeautifulSoup
import re
import json
import ast

In [23]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

url = 'https://www.sebi.gov.in/sebiweb/other/mutualfunds.jsp'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.maximize_window()

time.sleep(5)

#Click on the from date first. Icon xpath defined as date_picker
date_picker = driver.find_element_by_xpath('//*[@id="member-wrapper"]/section/div/section/div[1]/div/div[1]/div[1]/input')
ActionChains(driver).click(date_picker).perform()

number_of_months = 15

#Clicking the back button after 149 times till Jan 2010. Must select date 1
for d in range(number_of_months):
    previous = driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[1]/table/thead/tr[2]/th[1]')
    ActionChains(driver).click(previous).perform()
    
Start_date = driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[1]/table/tbody/tr[1]/td[6]')
ActionChains(driver).click(Start_date).perform()

Go_button = driver.find_element_by_xpath('//*[@id="member-wrapper"]/section/div/section/div[1]/div/a')
ActionChains(driver).click(Go_button).perform()

driver.implicitly_wait(20)

result = driver.page_source

# src = result.content
soup = BeautifulSoup(result, 'lxml')

data = soup.find_all('script')
data1 = soup.find_all('script', attrs={'type':'text/javascript'})

for item in data:
    if item not in data1:
        x = item

matches = re.search(r"xAxis.*\[\{(.|\n)*\}\]", str(x))
matches = re.sub(r'splitArea:(.|\n)*?}','',matches.group())
matches = re.sub(r'(\n|\r|\s|\t)','',matches)
matches = re.split(r'[a-zA-Z]+:',matches)
match_1 = matches[3].rstrip("}],")
match_2 = matches[-4].rstrip("},{]")
match_3 = matches[-1].lstrip("{ ").rstrip("}]")


date_list = ast.literal_eval(match_1+"]")
equity_list = ast.literal_eval(match_2+"]")
debt_list = ast.literal_eval(match_3+"]")



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\Priya Suresh\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


In [24]:
import pandas as pd

df = pd.DataFrame({"Date":date_list,"Equity":equity_list,"Debt":debt_list})
df

,Date,Equity,Debt
0,05-04-21,-228.67,2925.17
1,06-04-21,-87.08,2532.29
2,07-04-21,-55.87,-651.86
3,08-04-21,-54.38,1384.75
4,09-04-21,44.59,2355.77
...,...,...,...
337,14-07-22,-744.09,426.44
338,15-07-22,730.22,412.34
339,18-07-22,409.63,-446.37
340,19-07-22,-391.87,-397.87
